In [1]:
!pip install selenium

In [2]:
# 라이브러리 불러오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

import time

In [3]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib.request as req

from time import sleep

In [4]:
# 크롬 드라이버 잘 작동하는지 확인
driver = webdriver.Chrome() # 경로 지정해서 넣기
driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')

In [5]:
#2023년 esg등급 표 스크랩 함수
def esg_2023_scraper():
    driver = webdriver.Chrome()
    driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
    time.sleep(10) # 웹사이트가 켜지는 시간동안 잠시 멈춤 합니다.

    global data_df_2023
    data_df_2023 = pd.DataFrame(columns=("회사명", "KCGS_종합등급", "KCGS_환경", "KCGS_사회", "KCGS_지배구조", "한국ESG연구소", "Moody's", "MSCI", "S&P", "연도"))

    index = 0


    while index < 792: # 총 페이지가 792장 있기 떄문에 <792로 제한을 둡니다.
        # 각 페이지의 테이블 가져오기
        table = driver.find_element(By.ID,'gridtableeccbc87e4b5ce2fe28308fd9f2a7baf3')
        tbody = table.find_element(By.TAG_NAME, 'tbody')
        rows = tbody.find_elements(By.TAG_NAME, 'tr')


        # 한 페이지 속 기업들(rows)에 대해
        for i in range(len(rows)):

            rows = tbody.find_elements(By.TAG_NAME, 'tr')
            row = rows[i]
            # 기업명
            company_name = row.find_element(By.TAG_NAME, 'td').text
            # KCGS_종합 등급
            kcgs_1 = row.find_elements(By.TAG_NAME, 'td')[1].text
            # KCGS_환경 등급
            kcgs_2 = row.find_elements(By.TAG_NAME, 'td')[2].text
            # KCGS_사회 등급
            kcgs_3 = row.find_elements(By.TAG_NAME, 'td')[3].text
            # KCGS_지배구조 등급
            kcgs_4 = row.find_elements(By.TAG_NAME, 'td')[4].text
            #한국ESG연구소 등급
            kesg = row.find_elements(By.TAG_NAME, 'td')[5].text
            # Moody's 등급
            moodys = row.find_elements(By.TAG_NAME, 'td')[7].text
            # MSCI 등급
            msci = row.find_elements(By.TAG_NAME, 'td')[8].text
            # S&P 등급
            sp = row.find_elements(By.TAG_NAME, 'td')[9].text
            # 연도
            year = 2023


            # 기업별 추출한 정보를 dataframe의 한 행으로 저장
            data_df_2023.loc[index] = [company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, moodys, msci, sp, year]

            # index를 증가시키기 -> while이 반복되며 index가 792가 되면 종료
            index += 1

        # 한 페이지 내에는 10개의 row(기업) 존재,
        # 10개 기업 데이터 모두 추출한 후에는 다음 페이지로 이동
        if index % 10 == 0:
            driver.find_element(By.XPATH,'//*[@id="pagenavia87ff679a2f3e71d9181a67b7542122c"]/ul/li[13]/a').click()
            time.sleep(10)

    print(data_df_2023)
    return data_df_2023

In [27]:
#2022년 esg등급 표 스크랩 함수
def esg_2022_scraper():
    driver = webdriver.Chrome()
    driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
    time.sleep(10)
    
    # 연도를 2022년으로 바꾸고 정보 조회
    select = Select(driver.find_element(By.TAG_NAME, "select"))
    select.select_by_value('2022')
    button_element = driver.find_element(By.TAG_NAME, "button")
    button_element.click()
    time.sleep(10)

    data_df_2022 = pd.DataFrame(columns=("회사명", "KCGS_종합등급", "KCGS_환경", "KCGS_사회", "KCGS_지배구조", "한국ESG연구소", "Moody's", "MSCI", "S&P", "연도"))
    index = 0

    while index < 792:  # 총 페이지가 792장이라고 가정
        # 각 페이지의 테이블 가져오기
        table = driver.find_element(By.ID, 'gridtableeccbc87e4b5ce2fe28308fd9f2a7baf3')  # 실제 테이블의 ID로 수정
        tbody = table.find_element(By.TAG_NAME, 'tbody')
        rows = tbody.find_elements(By.TAG_NAME, 'tr')

        for i in range(len(rows)):
            row = rows[i]
            # 기업 정보 추출 (컬럼별로 요소 찾기)
            company_name = row.find_element(By.TAG_NAME, 'td').text
            kcgs_1 = row.find_elements(By.TAG_NAME, 'td')[1].text
            kcgs_2 = row.find_elements(By.TAG_NAME, 'td')[2].text
            kcgs_3 = row.find_elements(By.TAG_NAME, 'td')[3].text
            kcgs_4 = row.find_elements(By.TAG_NAME, 'td')[4].text
            kesg = row.find_elements(By.TAG_NAME, 'td')[5].text
            moodys = row.find_elements(By.TAG_NAME, 'td')[7].text
            msci = row.find_elements(By.TAG_NAME, 'td')[8].text
            sp = row.find_elements(By.TAG_NAME, 'td')[9].text
            year = 2022

            # 기업 정보를 데이터프레임에 추가
            data_df_2022.loc[index] = [company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, moodys, msci, sp, year]

            index += 1

        if index % 10 == 0:  # 한 페이지에 10개 기업 정보가 있음
            next_page_button = driver.find_element(By.XPATH, '//*[@id="pagenavia87ff679a2f3e71d9181a67b7542122c"]/ul/li[13]/a')
            next_page_button.click()
            time.sleep(10)  # 페이지 로딩을 위해 잠시 대기

    print(data_df_2022)
    return data_df_2022

In [15]:
#2021년 esg등급 표 스크랩 함수
def esg_2021_scraper():
    driver = webdriver.Chrome()
    driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
    time.sleep(10)
    
    # 연도를 2022년으로 바꾸고 정보 조회
    select = Select(driver.find_element(By.TAG_NAME, "select"))
    select.select_by_value('2021')
    button_element = driver.find_element(By.TAG_NAME, "button")
    button_element.click()
    time.sleep(10)

    data_df_2021 = pd.DataFrame(columns=("회사명", "KCGS_종합등급", "KCGS_환경", "KCGS_사회", "KCGS_지배구조", "한국ESG연구소", "Moody's", "MSCI", "S&P", "연도"))
    index = 0

    while index < 792:  # 총 페이지가 792장이라고 가정
        # 각 페이지의 테이블 가져오기
        table = driver.find_element(By.ID, 'gridtableeccbc87e4b5ce2fe28308fd9f2a7baf3')  # 실제 테이블의 ID로 수정
        tbody = table.find_element(By.TAG_NAME, 'tbody')
        rows = tbody.find_elements(By.TAG_NAME, 'tr')

        for i in range(len(rows)):
            row = rows[i]
            # 기업 정보 추출 (컬럼별로 요소 찾기)
            company_name = row.find_element(By.TAG_NAME, 'td').text
            kcgs_1 = row.find_elements(By.TAG_NAME, 'td')[1].text
            kcgs_2 = row.find_elements(By.TAG_NAME, 'td')[2].text
            kcgs_3 = row.find_elements(By.TAG_NAME, 'td')[3].text
            kcgs_4 = row.find_elements(By.TAG_NAME, 'td')[4].text
            kesg = row.find_elements(By.TAG_NAME, 'td')[5].text
            moodys = row.find_elements(By.TAG_NAME, 'td')[7].text
            msci = row.find_elements(By.TAG_NAME, 'td')[8].text
            sp = row.find_elements(By.TAG_NAME, 'td')[9].text
            year = 2021

            # 기업 정보를 데이터프레임에 추가
            data_df_2021.loc[index] = [company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, moodys, msci, sp, year]

            index += 1

        if index % 10 == 0:  # 한 페이지에 10개 기업 정보가 있음
            next_page_button = driver.find_element(By.XPATH, '//*[@id="pagenavia87ff679a2f3e71d9181a67b7542122c"]/ul/li[13]/a')
            next_page_button.click()
            time.sleep(10)  # 페이지 로딩을 위해 잠시 대기

    print(data_df_2021)
    return data_df_2021

In [19]:
#2020년 esg등급 표 스크랩 함수
def esg_2020_scraper():
    driver = webdriver.Chrome()
    driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
    time.sleep(10)
    
    # 연도를 2020년으로 바꾸고 정보 조회
    select = Select(driver.find_element(By.TAG_NAME, "select"))
    select.select_by_value('2020')
    button_element = driver.find_element(By.TAG_NAME, "button")
    button_element.click()
    time.sleep(10)

    data_df_2020 = pd.DataFrame(columns=("회사명", "KCGS_종합등급", "KCGS_환경", "KCGS_사회", "KCGS_지배구조", "한국ESG연구소", "Moody's", "MSCI", "S&P", "연도"))
    index = 0

    while index < 792:  # 총 페이지가 792장이라고 가정
        # 각 페이지의 테이블 가져오기
        table = driver.find_element(By.ID, 'gridtableeccbc87e4b5ce2fe28308fd9f2a7baf3')  # 실제 테이블의 ID로 수정
        tbody = table.find_element(By.TAG_NAME, 'tbody')
        rows = tbody.find_elements(By.TAG_NAME, 'tr')

        for i in range(len(rows)):
            row = rows[i]
            # 기업 정보 추출 (컬럼별로 요소 찾기)
            company_name = row.find_element(By.TAG_NAME, 'td').text
            kcgs_1 = row.find_elements(By.TAG_NAME, 'td')[1].text
            kcgs_2 = row.find_elements(By.TAG_NAME, 'td')[2].text
            kcgs_3 = row.find_elements(By.TAG_NAME, 'td')[3].text
            kcgs_4 = row.find_elements(By.TAG_NAME, 'td')[4].text
            kesg = row.find_elements(By.TAG_NAME, 'td')[5].text
            moodys = row.find_elements(By.TAG_NAME, 'td')[7].text
            msci = row.find_elements(By.TAG_NAME, 'td')[8].text
            sp = row.find_elements(By.TAG_NAME, 'td')[9].text
            year = 2020

            # 기업 정보를 데이터프레임에 추가
            data_df_2020.loc[index] = [company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, moodys, msci, sp, year]

            index += 1

        if index % 10 == 0:  # 한 페이지에 10개 기업 정보가 있음
            next_page_button = driver.find_element(By.XPATH, '//*[@id="pagenavia87ff679a2f3e71d9181a67b7542122c"]/ul/li[13]/a')
            next_page_button.click()
            time.sleep(10)  # 페이지 로딩을 위해 잠시 대기

    print(data_df_2020)
    return data_df_2020

In [9]:
esg_2023_scraper()

           회사명 KCGS_종합등급 KCGS_환경 KCGS_사회 KCGS_지배구조 한국ESG연구소 Moody's MSCI S&P  \
0       AJ네트웍스        B+       C      B+        B+        -       -    -   -   
1        AK홀딩스        B+       B       A         B        -       -    -   4   
2          BGF        B+       A      A+         B        -       -    -  18   
3       BGF리테일         A       A      A+         A        A      23  BBB  58   
4      BNK금융지주         A       A       A         A        A      37    A  43   
..         ...       ...     ...     ...       ...      ...     ...  ...  ..   
795      효성티앤씨         A       A      A+        B+        A       -    -  27   
796       효성화학         A       A      A+        B+        -       -    -  29   
797         후성         B       B      B+         B       B+       -    -   7   
798      휠라홀딩스        B+       B       A        B+        A      24   BB  40   
799  휴니드테크놀러지스         D       D       C         C        -       -    -   -   

       연도  
0    2023  
1    2023  
2  

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2023
1,AK홀딩스,B+,B,A,B,-,-,-,4,2023
2,BGF,B+,A,A+,B,-,-,-,18,2023
3,BGF리테일,A,A,A+,A,A,23,BBB,58,2023
4,BNK금융지주,A,A,A,A,A,37,A,43,2023
...,...,...,...,...,...,...,...,...,...,...
795,효성티앤씨,A,A,A+,B+,A,-,-,27,2023
796,효성화학,A,A,A+,B+,-,-,-,29,2023
797,후성,B,B,B+,B,B+,-,-,7,2023
798,휠라홀딩스,B+,B,A,B+,A,24,BB,40,2023


In [30]:
esg_2022_scraper()

           회사명 KCGS_종합등급 KCGS_환경 KCGS_사회 KCGS_지배구조 한국ESG연구소 Moody's MSCI S&P  \
0       AJ네트웍스        B+       C      B+        B+        -       -    -   -   
1        AK홀딩스        B+       B       A         B        -       -    -   4   
2          BGF        B+       A      A+         B        -       -    -  18   
3       BGF리테일         A       A      A+         A        A      23  BBB  58   
4      BNK금융지주         A       A       A         A        A      37    A  43   
..         ...       ...     ...     ...       ...      ...     ...  ...  ..   
789  휴니드테크놀러지스         D       D       C         C        -       -    -   -   
790        휴비스        B+      B+       A        B+        -       -    -   6   
791        휴스틸         D       D       D         C        -       -    -   -   
792       흥국화재        B+       C      B+        B+        -       -    -   -   
793       흥아해운         C       C       B         C        -       -    -   -   

       연도  
0    2022  
1    2022  
2  

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2022
1,AK홀딩스,B+,B,A,B,-,-,-,4,2022
2,BGF,B+,A,A+,B,-,-,-,18,2022
3,BGF리테일,A,A,A+,A,A,23,BBB,58,2022
4,BNK금융지주,A,A,A,A,A,37,A,43,2022
...,...,...,...,...,...,...,...,...,...,...
789,휴니드테크놀러지스,D,D,C,C,-,-,-,-,2022
790,휴비스,B+,B+,A,B+,-,-,-,6,2022
791,휴스틸,D,D,D,C,-,-,-,-,2022
792,흥국화재,B+,C,B+,B+,-,-,-,-,2022


In [18]:
esg_2021_scraper()

           회사명 KCGS_종합등급 KCGS_환경 KCGS_사회 KCGS_지배구조 한국ESG연구소 Moody's MSCI S&P  \
0       AJ네트웍스         B       D       B         B        -       -    -   -   
1        AK홀딩스        B+       B      B+        B+        -       -    -   1   
2          BGF         A       A      A+         A        -       -    -  13   
3       BGF리테일         A       A      A+         A        S       -  BBB  11   
4      BNK금융지주        A+       A      A+        A+        A       -    A  38   
..         ...       ...     ...     ...       ...      ...     ...  ...  ..   
789  휴니드테크놀러지스         B       C      B+        B+        -       -    -   -   
790        휴비스         A      B+      A+         A        -       -    -   5   
791        휴스틸         C       C       C         B        -       -    -   -   
792       흥국화재        B+       B       A         B        -       -    -   -   
793       흥아해운         D       D       C         D        -       -    -   -   

       연도  
0    2021  
1    2021  
2  

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B,D,B,B,-,-,-,-,2021
1,AK홀딩스,B+,B,B+,B+,-,-,-,1,2021
2,BGF,A,A,A+,A,-,-,-,13,2021
3,BGF리테일,A,A,A+,A,S,-,BBB,11,2021
4,BNK금융지주,A+,A,A+,A+,A,-,A,38,2021
...,...,...,...,...,...,...,...,...,...,...
789,휴니드테크놀러지스,B,C,B+,B+,-,-,-,-,2021
790,휴비스,A,B+,A+,A,-,-,-,5,2021
791,휴스틸,C,C,C,B,-,-,-,-,2021
792,흥국화재,B+,B,A,B,-,-,-,-,2021


In [21]:
esg_2020_scraper()

           회사명 KCGS_종합등급 KCGS_환경 KCGS_사회 KCGS_지배구조 한국ESG연구소 Moody's MSCI S&P  \
0       AJ네트웍스         B       D       B         B        -       -    -   -   
1        AK홀딩스        B+       C      B+        B+        -       -    -   -   
2          BGF        B+       B       A        B+        -       -    -   -   
3       BGF리테일         A      B+      B+         A        -       -   BB   -   
4      BNK금융지주        A+      B+      A+        A+        -       -    A   -   
..         ...       ...     ...     ...       ...      ...     ...  ...  ..   
789  휴니드테크놀러지스         C       C       B         B        -       -    -   -   
790        휴비스        B+      B+      B+        B+        -       -    -   -   
791        휴스틸         C       B       B         C        -       -    -   -   
792       흥국화재         B       D       A         B        -       -    -   -   
793       흥아해운         D       D       C         D        -       -    -   -   

       연도  
0    2020  
1    2020  
2  

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B,D,B,B,-,-,-,-,2020
1,AK홀딩스,B+,C,B+,B+,-,-,-,-,2020
2,BGF,B+,B,A,B+,-,-,-,-,2020
3,BGF리테일,A,B+,B+,A,-,-,BB,-,2020
4,BNK금융지주,A+,B+,A+,A+,-,-,A,-,2020
...,...,...,...,...,...,...,...,...,...,...
789,휴니드테크놀러지스,C,C,B,B,-,-,-,-,2020
790,휴비스,B+,B+,B+,B+,-,-,-,-,2020
791,휴스틸,C,B,B,C,-,-,-,-,2020
792,흥국화재,B,D,A,B,-,-,-,-,2020


In [38]:
data_df_2022 = esg_2022_scraper()
data_df_2021 = esg_2021_scraper()
data_df_2020 = esg_2020_scraper()

data_df_2023.to_csv('esg_2023.csv')
data_df_2022.to_csv('esg_2022.csv')
data_df_2021.to_csv('esg_2021.csv')
data_df_2020.to_csv('esg_2020.csv')

           회사명 KCGS_종합등급 KCGS_환경 KCGS_사회 KCGS_지배구조 한국ESG연구소 Moody's MSCI S&P  \
0       AJ네트웍스        B+       C      B+        B+        -       -    -   -   
1        AK홀딩스        B+       B       A         B        -       -    -   4   
2          BGF        B+       A      A+         B        -       -    -  18   
3       BGF리테일         A       A      A+         A        A      23  BBB  58   
4      BNK금융지주         A       A       A         A        A      37    A  43   
..         ...       ...     ...     ...       ...      ...     ...  ...  ..   
789  휴니드테크놀러지스         D       D       C         C        -       -    -   -   
790        휴비스        B+      B+       A        B+        -       -    -   6   
791        휴스틸         D       D       D         C        -       -    -   -   
792       흥국화재        B+       C      B+        B+        -       -    -   -   
793       흥아해운         C       C       B         C        -       -    -   -   

       연도  
0    2022  
1    2022  
2  

In [39]:
# 데이터 합치기
data_df_merge = pd.concat([data_df_2023, data_df_2022, data_df_2021, data_df_2020])
data_df_merge

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2023
1,AK홀딩스,B+,B,A,B,-,-,-,4,2023
2,BGF,B+,A,A+,B,-,-,-,18,2023
3,BGF리테일,A,A,A+,A,A,23,BBB,58,2023
4,BNK금융지주,A,A,A,A,A,37,A,43,2023
...,...,...,...,...,...,...,...,...,...,...
789,휴니드테크놀러지스,C,C,B,B,-,-,-,-,2020
790,휴비스,B+,B+,B+,B+,-,-,-,-,2020
791,휴스틸,C,B,B,C,-,-,-,-,2020
792,흥국화재,B,D,A,B,-,-,-,-,2020


In [40]:
# 정렬하기
data_df_merge.sort_values(by=['회사명', '연도'], inplace = True)
data_df_merge

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B,D,B,B,-,-,-,-,2020
0,AJ네트웍스,B,D,B,B,-,-,-,-,2021
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2022
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2023
1,AK홀딩스,B+,C,B+,B+,-,-,-,-,2020
...,...,...,...,...,...,...,...,...,...,...
792,흥국화재,B+,B,A,B,-,-,-,-,2021
792,흥국화재,B+,C,B+,B+,-,-,-,-,2022
793,흥아해운,D,D,C,D,-,-,-,-,2020
793,흥아해운,D,D,C,D,-,-,-,-,2021


In [41]:
# 저장하기
data_df_merge.to_csv('esg_merge.csv', encoding='utf-8-sig')